# wos 데이터 살펴보기
- klue 내 DST(Dialogue State Tracking) task를 이해하기 위함
- DST task에 사용되는 데이터셋은 wos dataset이며, klue에서 한글버전으로 만들어짐
- dst task를 구성하기 위해 어떻게 input data를 구성하는지 살펴봄
- klue-base line 기준

In [4]:
import argparse
import json
import logging
import os

In [32]:
from transformers import AutoTokenizer
from klue_baseline import KLUE_TASKS
from klue_baseline.data import DataProcessor, KlueDataModule, WoSProcessor
from klue_baseline.models import BaseTransformer, Mode, DPTransformer, DSTTransformer
from klue_baseline.utils import Command
from klue_baseline.metrics import WoS_JGA, WoS_SlotMicroF1

## Initialize

In [6]:
args = argparse.Namespace(
    accumulate_grad_batches=1, 
    adafactor=False, 
    adam_epsilon=1e-08, 
    attention_dropout=None, 
    cache_dir='', 
    command='train', 
    config_name='', 
    data_dir='./wos', 
    decoder_layerdrop=None, 
    dev_file_name=None, 
    dropout=None, 
    early_stopping_mode='max', 
    encoder_layerdrop=None, 
    eval_batch_size=16, 
    fp16=False, 
    gpus=None, 
    gradient_clip_val=1.0, 
    learning_rate=3e-05, 
    lr_scheduler='linear', 
    max_epochs=5, 
    max_seq_length=510, 
    metric_key='slot_micro_f1', 
    model_name_or_path='./monologg/koelectra-base-v3-discriminator', 
    num_sanity_val_steps=2, 
    num_workers=4, 
    ontology_path='', 
    output_dir='./wos/wos_output', 
    parallel_decoding=True, 
    patience=100000, 
    seed=42, 
    task='wos', 
    teacher_forcing=0.5, 
    test_file_name=None, 
    tokenizer_name=None, 
    tpu_cores=None, 
    train_batch_size=16, 
    train_file_name=None, 
    truncate=True, 
    verbose_step_count=100, 
    warmup_ratio=0.1, 
    warmup_steps=None, 
    weight_decay=0.01
)

## vocab file load

In [68]:
f = open('./vocab.txt', 'r', encoding='utf8')
text = f.readlines()
text = [i.replace('\n','') for i in text]

In [78]:
with open('./wos/ontology.json', encoding='utf8') as json_file:
    json_data = json.load(json_file)

In [88]:
print('미리 ontology로 정의된 state', list(json_data)[0:5], end='\n\n')
print('ontology 예시 : 관광-종류', json_data['관광-종류'])

미리 ontology로 정의된 state ['관광-경치 좋은', '관광-교육적', '관광-도보 가능', '관광-문화 예술', '관광-역사적']

ontology 예시 : 관광-종류 ['none', 'dontcare', '공원', '박물관', '공연', '즐길거리', '랜드마크', '쇼핑', '여행자 자료', '기타', '놀이공원', '동물원', '교통편']


## dataloader make

In [23]:
task = KlueTask(
        WoSProcessor,
        DSTTransformer,
        {"joint_goal_acc": WoS_JGA, "slot_micro_f1": WoS_SlotMicroF1}
)

task.setup(args, args.command)

In [35]:
task.train_loader.dataset.features[0].guid

'wos-v1_train_00000-0'

In [ ]:
'''
"guid": "wos-v1_train_00000",
"domains": [
    "관광",
    "식당"
],
"dialogue": [
    {
        "role": "user",
        "text": "서울 중앙에 있는 박물관을 찾아주세요",
        "state": [
            "관광-종류-박물관",
            "관광-지역-서울 중앙"
        ]
    },
    {
        "role": "sys",
        "text": "안녕하세요. 문화역서울 284은 어떠신가요? 평점도 4점으로 방문객들에게 좋은 평가를 받고 있습니다."
    },
    {
        "role": "user",
        "text": "좋네요 거기 평점은 말해주셨구 전화번호가 어떻게되나요?",
        "state": [
            "관광-종류-박물관",
            "관광-지역-서울 중앙",
            "관광-이름-문화역서울 284"
        ]
    },
    {
        "role": "sys",
        "text": "전화번호는 983880764입니다. 더 필요하신 게 있으실까요?"
    }
'''

In [92]:
print('BERT input')
print('input id', task.train_loader.dataset.features[0].input_id)
print('input to vocab', [text[i] for i in task.train_loader.dataset.features[0].input_id])

BERT input
input id [2, 3, 3, 6265, 6672, 4073, 3249, 4034, 8732, 4292, 6722, 4076, 8553, 3]
input to vocab ['[CLS]', '[SEP]', '[SEP]', '서울', '중앙', '##에', '있', '##는', '박물관', '##을', '찾아', '##주', '##세요', '[SEP]']


### 맞추어야 할 정답 slot

In [89]:
print('미리 ontology로 정의된 슬롯 45개 중 정답 slot 예측')
print('gating ids', task.train_loader.dataset.features[0].gating_id)

미리 ontology로 정의된 슬롯 45개 중 정답 state가 있는 위치
gating id [0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
target_ids_0 = []
for slot in task.train_loader.dataset.features[0].target_ids:
    slot_token = []
    for token in slot :
        slot_token.append(text[token])
    target_ids_0.append(slot_token)

### 맞추어야 할 token slot

In [99]:
print('미리 ontology로 정의된 슬롯 45개 중 정답 token 예측')
print('target ids', task.train_loader.dataset.features[0].target_ids, end='\n\n')
print('target ids to vocab', target_ids_0)

미리 ontology로 정의된 슬롯 45개 중 정답 token 예측
target ids [[21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [8732, 3, 0], [21832, 11764, 3], [6265, 6672, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3], [21832, 11764, 3]]

target ids to vocab [['no', '##ne', '[SEP]'], ['no', '##ne', '[SEP]'], ['no', '##ne', '[SEP]'], ['no'